In [1]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\megha\anaconda3\envs\openaidambo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Extract data from PDF FILE
def load_pdf_file(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents

In [8]:
extracted_data=load_pdf_file(data="../Data/")

In [ ]:
#Chunking
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

5859


In [11]:
#Embedding
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings=download_hugging_face_embeddings()

C:\Users\megha\AppData\Local\Temp\ipykernel_23592\4283445018.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
query=embeddings.embed_query("hello world")
len(query)

384

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
import os

In [17]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [20]:
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="medibot",
    embedding=embeddings
)

In [29]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medibot",
    embedding=embeddings
)

In [30]:
docsearch

In [24]:
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")

In [25]:
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [37]:
#integrating llm
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [28]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three to five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [38]:
qa_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,qa_chain)

In [39]:
res=rag_chain.invoke({"input":"how to cure  Acromegaly and gigantism?"})
print(res["answer"])

Acromegaly and gigantism are caused by the abnormal release of growth hormone from the pituitary gland. While the provided text details the definition and diagnosis of these conditions, it does not specify a cure. Treatment options, therefore, are not discussed in this context.
